## Step 1: Mount Google Drive and Clone Repository

In [1]:
from google.colab import drive

drive.mount('/content/drive')
print("✓ Google Drive mounted")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted


In [3]:
import os
os.chdir('/content')

!rm -rf automatic-pain-recognition
!git clone https://github.com/alicka33/automatic-pain-recognition.git
os.chdir('/content/automatic-pain-recognition')
print(f"✓ Repository cloned. Current directory: {os.getcwd()}")

Cloning into 'automatic-pain-recognition'...
remote: Enumerating objects: 562, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 562 (delta 115), reused 197 (delta 51), pack-reused 277 (from 1)
Receiving objects: 100% (562/562), 23.63 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (239/239), done.
✓ Repository cloned. Current directory: /content/automatic-pain-recognition


In [ ]:
import subprocess
import sys

!pip install -q -r requirements.txt

print("✓ Requirements installed")


✓ Requirements installed


## Step 2: Add Project to Python Path

In [ ]:
import os
import subprocess
import sys

print("Cleaning and reinstalling MediaPipe for Colab...")

!pip uninstall -y mediapipe
!rm -rf /usr/local/lib/python3.12/dist-packages/mediapipe
!pip install --no-cache-dir mediapipe==0.10.14

print("✓ MediaPipe 0.10.14 installed, restarting kernel...")

os.kill(os.getpid(), 9)


In [4]:
import sys
from pathlib import Path
import os

project_root = Path('/content/automatic-pain-recognition')
server_dir = project_root / 'pain_detection_app_server'

for p in (project_root, server_dir):
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))

os.chdir(server_dir)
print(f"✓ Working directory set to: {os.getcwd()}")
print(f"✓ Paths added to sys.path")

✓ Working directory set to: /content/automatic-pain-recognition/pain_detection_app_server
✓ Paths added to sys.path


## Step 3: Test All Components

Run each test below. ✓ = pass, ✗ = fail.

In [5]:
print("\n" + "="*70)
print("TEST 1: Import Core Libraries")
print("="*70)

tests_passed = 0
tests_total = 0

# PyTorch
try:
    import torch
    print(f"✓ PyTorch {torch.__version__}")
    print(f"  CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
    tests_passed += 1
except ImportError as e:
    print(f"✗ PyTorch: {e}")
tests_total += 1

# NumPy
try:
    import numpy as np
    print(f"✓ NumPy {np.__version__}")
    tests_passed += 1
except ImportError as e:
    print(f"✗ NumPy: {e}")
tests_total += 1

# OpenCV
try:
    import cv2
    print(f"✓ OpenCV {cv2.__version__}")
    tests_passed += 1
except ImportError as e:
    print(f"✗ OpenCV: {e}")
tests_total += 1

# MediaPipe
try:
    import mediapipe as mp
    print(f"✓ MediaPipe {mp.__version__}")
    tests_passed += 1
except ImportError as e:
    print(f"✗ MediaPipe: {e}")
tests_total += 1

# FastAPI
try:
    import fastapi
    print(f"✓ FastAPI {fastapi.__version__}")
    tests_passed += 1
except ImportError as e:
    print(f"✗ FastAPI: {e}")
tests_total += 1

# scikit-learn
try:
    import sklearn
    print(f"✓ scikit-learn {sklearn.__version__}")
    tests_passed += 1
except ImportError as e:
    print(f"✗ scikit-learn: {e}")
tests_total += 1

print(f"\nResult: {tests_passed}/{tests_total} library imports successful")


TEST 1: Import Core Libraries
✓ PyTorch 2.9.0+cpu
  CUDA available: False
✓ NumPy 2.0.2
✓ OpenCV 4.12.0
✓ MediaPipe 0.10.14
✓ FastAPI 0.123.10
✓ scikit-learn 1.6.1

Result: 6/6 library imports successful


In [6]:
print("\n" + "="*70)
print("TEST 2: Configuration File")
print("="*70)

try:
    import config
    print(f"✓ Config loaded successfully")
    print(f"\n  Model Configuration:")
    print(f"    Model Type: {config.MODEL_TYPE}")
    print(f"    Num Classes: {config.NUM_CLASSES}")
    print(f"    Num Features: {config.NUM_FEATURES}")
    print(f"    Hidden Size: {config.HIDDEN_SIZE}")
    print(f"    Max Sequence Length: {config.MAX_SEQUENCE_LENGTH}")
    print(f"\n  Path Configuration:")
    print(f"    Model Checkpoint: {config.MODEL_CHECKPOINT_PATH}")
    print(f"    Data Dir: {config.DATA_DIR}")
    tests_passed += 1
except Exception as e:
    print(f"✗ Config error: {e}")
tests_total += 1

print(f"\nResult: Config test {'PASSED' if tests_passed == tests_total else 'FAILED'}")


TEST 2: Configuration File
✓ Config loaded successfully

  Model Configuration:
    Model Type: sta_lstm
    Num Classes: 2
    Num Features: 300
    Hidden Size: 128
    Max Sequence Length: 200

  Path Configuration:
    Model Checkpoint: /content/automatic-pain-recognition/pain_detection_app_server/model/model_paths/testing_new_code_sta_lstm_2_classes_300_coord.pt
    Data Dir: /content/automatic-pain-recognition/pain_detection_app_server/data

Result: Config test PASSED


In [ ]:
print("\n" + "="*70)
print("TEST 3: Model File Existence")
print("="*70)

import config
from pathlib import Path

tests_total += 1

if config.MODEL_CHECKPOINT_PATH.exists():
    size_mb = config.MODEL_CHECKPOINT_PATH.stat().st_size / (1024 * 1024)
    print(f"✓ Model checkpoint found")
    print(f"  Path: {config.MODEL_CHECKPOINT_PATH}")
    print(f"  Size: {size_mb:.2f} MB")
    tests_passed += 1
else:
    print(f"✗ Model checkpoint NOT found")
    print(f"  Expected at: {config.MODEL_CHECKPOINT_PATH}")
    print(f"  Add your trained model file there")

print(f"\nOptional Files:")
if config.REFERENCE_KEYPOINTS_PATH.exists():
    print(f"  ✓ Reference keypoints found")
else:
    print(f"  ✗ Reference keypoints not found (frontalization disabled)")

if config.LANDMARK_INDICES_PATH.exists():
    print(f"  ✓ Landmark indices found")
else:
    print(f"  ✗ Landmark indices not found (using all landmarks)")

print(f"\nResult: Model file test {'PASSED' if tests_passed == tests_total else 'FAILED'}")


TEST 3: Model File Existence
✓ Model checkpoint found
  Path: /content/automatic-pain-recognition/pain_detection_app_server/model/model_paths/testing_new_code_sta_lstm_2_classes_300_coord.pt
  Size: 3.62 MB

Optional Files:
  ✓ Reference keypoints found
  ✓ Landmark indices found

Result: Model file test PASSED


In [8]:
print("\n" + "="*70)
print("TEST 4: Video Processing Pipeline")
print("="*70)

tests_total += 1
try:
    from services.processing_pipeline_mediapipe import (
        create_face_mesh,
        parse_landmarks_from_results
    )
    print("✓ MediaPipe pipeline imports successful")

    face_mesh = create_face_mesh()
    print("✓ FaceMesh instance created")
    face_mesh.close()
    print("✓ FaceMesh closed properly")
    tests_passed += 1
except Exception as e:
    print(f"✗ Video processing error: {e}")
    import traceback
    traceback.print_exc()

print(f"\nResult: Video processing test {'PASSED' if tests_passed == tests_total else 'FAILED'}")


TEST 4: Video Processing Pipeline
✓ MediaPipe pipeline imports successful
✓ FaceMesh instance created
✓ FaceMesh closed properly

Result: Video processing test PASSED


In [9]:
print("\n" + "="*70)
print("TEST 5: Inference Module Imports")
print("="*70)

tests_total += 1
try:
    from services.video_inference import VideoInferenceHelper, load_model_for_inference
    print("✓ VideoInferenceHelper import successful")
    print("✓ load_model_for_inference import successful")
    tests_passed += 1
except Exception as e:
    print(f"✗ Inference module error: {e}")
    import traceback
    traceback.print_exc()

print(f"\nResult: Inference module test {'PASSED' if tests_passed == tests_total else 'FAILED'}")


TEST 5: Inference Module Imports
✓ VideoInferenceHelper import successful
✓ load_model_for_inference import successful

Result: Inference module test PASSED


In [10]:
print("\n" + "="*70)
print("TEST 6: Model Loading")
print("="*70)

import config

tests_total += 1

if not config.MODEL_CHECKPOINT_PATH.exists():
    print(f"✗ Skipping (no model checkpoint found)")
    print(f"  Add your trained model to: {config.MODEL_CHECKPOINT_PATH}")
else:
    try:
        import torch
        from services.video_inference import load_model_for_inference

        if config.MODEL_TYPE == "attention_lstm":
            from model.Attention_LSTM import AttentionSequenceModel as ModelClass
        elif config.MODEL_TYPE == "bi_lstm":
            from model.Bi_LSTM import SequenceModel as ModelClass
        elif config.MODEL_TYPE == "sta_lstm":
            from model.STA_LSTM import STA_LSTM as ModelClass
        else:
            raise ValueError(f"Unknown model type: {config.MODEL_TYPE}")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"  Loading model on device: {device}")

        model = load_model_for_inference(
            model_class=ModelClass,
            model_path=str(config.MODEL_CHECKPOINT_PATH),
            device=device,
            input_size=config.NUM_FEATURES,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            num_classes=config.NUM_CLASSES,
            dropout_prob=config.DROPOUT_PROB
        )

        num_params = sum(p.numel() for p in model.parameters())
        print(f"✓ Model loaded successfully")
        print(f"  Model Type: {config.MODEL_TYPE}")
        print(f"  Device: {device}")
        print(f"  Parameters: {num_params:,}")
        tests_passed += 1
    except Exception as e:
        print(f"✗ Model loading error: {e}")
        import traceback
        traceback.print_exc()

print(f"\nResult: Model loading test {'PASSED' if tests_passed == tests_total else 'FAILED'}")


TEST 6: Model Loading
  Loading model on device: cpu
✓ Model loaded successfully
  Model Type: sta_lstm
  Device: cpu
  Parameters: 946,863

Result: Model loading test PASSED


## Final Summary

In [11]:
print("\n" + "="*70)
print("FINAL TEST SUMMARY")
print("="*70)

import config
model_present = config.MODEL_CHECKPOINT_PATH.exists()
summary = {
    "Core Libraries": True,
    "Configuration": True,
    "Video Processing": True,
    "Inference Module": True,
    "Model Checkpoint Present": model_present,
}

for test_name, passed in summary.items():
    status = "PASS" if passed else "MISSING"
    symbol = "✓" if passed else "✗"
    print(f"{symbol} {test_name:40} {status}")

print("\n" + "="*70)
if model_present:
    print("All checks passed (model found). Server ready to run.")
    print("Next: run `python app.py` or `uvicorn app:app --reload --port 7860`")
else:
    print("Model checkpoint missing. Add your trained model and re-run Test 6.")
print("="*70)



FINAL TEST SUMMARY
✓ Core Libraries                           PASS
✓ Configuration                            PASS
✓ Video Processing                         PASS
✓ Inference Module                         PASS
✓ Model Checkpoint Present                 PASS

All checks passed (model found). Server ready to run.
Next: run `python app.py` or `uvicorn app:app --reload --port 7860`
